# 🚀 Depth Estimation с Depth Anything V3 + Segmentation

**Обновлённая версия с Depth Anything V3!**

Этот notebook:
1. **Использует Depth Anything V3 (DA3NESTED)** - автоматическое определение camera intrinsics!
2. **Mask R-CNN** для сегментации кубов и параллелепипедов
3. **Вычисляет расстояния** до каждого объекта в метрах
4. **Определяет связи** между параллелепипедами и кубами
5. **Визуализирует** результаты с depth map, confidence и расстояниями

## 🎯 Преимущества V3:

- ✅ **Автоматически оценивает camera intrinsics** (не нужно передавать из Unity!)
- ✅ **Depth сразу в метрах** (не нужна формула конвертации)
- ✅ **Confidence map** для фильтрации ненадёжных областей
- ✅ **Лучшая точность** по сравнению с V2
- ✅ **Multi-view support** (опционально)

## 1. Установка зависимостей

In [ ]:
# Depth Anything V3
!pip install xformers torch>=2 torchvision
!pip install git+https://github.com/ByteDance-Seed/Depth-Anything-3.git

# Segmentation
!pip install pycocotools
!pip install opencv-python-headless
!pip install matplotlib
!pip install scipy
!pip install pillow

In [ ]:
import os
import json
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
from collections import defaultdict
import math

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision.models.detection import maskrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

from depth_anything_3.api import DepthAnything3

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 2. Конфигурация

In [ ]:
# --- ЛОГИКА ЗАГРУЗКИ ДАТАСЕТА ---
import os
import subprocess
import sys

REPO_URL = "https://github.com/SergKurchev/strawberry_peduncle_segmentation.git"
REPO_NAME = "strawberry_peduncle_segmentation"
KAGGLE_PATH = "/kaggle/input/strawberry-peduncle-segmentation"

# 1. Проверяем наличие на Kaggle (если запуск там)
if os.path.exists(KAGGLE_PATH):
    DATASET_PATH = KAGGLE_PATH
    print(f"✅ Используется датасет из Kaggle Input: {DATASET_PATH}")
else:
    # 2. Если на Kaggle нет, клонируем репозиторий из GitHub
    if not os.path.exists(REPO_NAME):
        print(f"🚀 Клонируем репозиторий из GitHub: {REPO_URL}...")
        subprocess.run(["git", "clone", REPO_URL])
    else:
        print(f"✅ Репозиторий {REPO_NAME} уже существует.")
    
    # Определение пути к датасету внутри репо
    # Проверяем два варианта структуры (с вложенной папкой и без)
    opt1 = os.path.join(REPO_NAME, "strawberry_peduncle_segmentation", "dataset")
    opt2 = os.path.join(REPO_NAME, "dataset")
    
    if os.path.exists(opt1):
        DATASET_PATH = opt1
    elif os.path.exists(opt2):
        DATASET_PATH = opt2
    else:
        DATASET_PATH = REPO_NAME
        print(f"⚠️ Внимание: Папка 'dataset' не найдена, используем корень репозитория.")

print(f"📍 Итоговый путь к датасету: {DATASET_PATH}")

IMAGES_PATH = os.path.join(DATASET_PATH, "images")
MASKS_PATH = os.path.join(DATASET_PATH, "masks")
ANNOTATIONS_PATH = os.path.join(DATASET_PATH, "annotations.json")

# Проверка наличия изображений
if not os.path.exists(IMAGES_PATH):
    print(f"❌ ВНИМАНИЕ: Папка с изображениями не найдена по пути: {IMAGES_PATH}")
    print(f"   Если вы используете GitHub, убедитесь, что изображения добавлены в репозиторий.")
    print(f"   Если изображения на Kaggle, добавьте датасет в Input.")

# Гиперпараметры
NUM_CLASSES = 3  # background + red_cube + green_parallelepiped
BATCH_SIZE = 4
NUM_EPOCHS = 30
LEARNING_RATE = 0.005
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Категории
CATEGORIES = {
    0: "background",
    1: "red_cube",
    2: "green_parallelepiped"
}

# Размер обучающей выборки
TRAIN_RATIO = 0.8


## 3. Загрузка моделей

In [ ]:
# Инициализация Depth Anything V3
print(f"Loading Depth Anything V3: {DEPTH_MODEL_NAME}")

depth_model = DepthAnything3.from_pretrained(DEPTH_MODEL_NAME)
depth_model = depth_model.to(DEVICE)

print("✅ Depth Anything V3 loaded")
print(f"   Model: {DEPTH_MODEL_NAME}")
print(f"   Features: Auto camera intrinsics, metric depth, confidence map")

In [ ]:
# Загрузка Mask R-CNN для сегментации
def get_segmentation_model(num_classes):
    model = maskrcnn_resnet50_fpn(weights=None)
    
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask, hidden_layer, num_classes)
    
    return model

segmentation_model = get_segmentation_model(num_classes=3)
segmentation_model.load_state_dict(torch.load(SEGMENTATION_MODEL_PATH, map_location='cpu'))
segmentation_model = segmentation_model.to(DEVICE).eval()

print("✅ Mask R-CNN loaded")

## 4. Функции для обработки

In [ ]:
def predict_depth_v3(image_rgb):
    """
    Предсказывает depth map в метрах используя Depth Anything V3.
    
    Args:
        image_rgb: RGB изображение (numpy array или PIL Image)
    
    Returns:
        result: dict с depth, confidence, intrinsics
    """
    # Конвертация в PIL если нужно
    if isinstance(image_rgb, np.ndarray):
        image_pil = Image.fromarray(image_rgb)
    else:
        image_pil = image_rgb
    
    with torch.no_grad():
        # V3 принимает список изображений
        prediction = depth_model.inference([image_pil])
    
    # Извлекаем результаты
    depth = prediction.depth[0]  # (H, W) в МЕТРАХ!
    conf = prediction.conf[0] if hasattr(prediction, 'conf') else None  # (H, W) confidence
    intrinsics = prediction.intrinsics[0] if hasattr(prediction, 'intrinsics') else None  # (3, 3)
    
    return {
        'depth': depth,
        'confidence': conf,
        'intrinsics': intrinsics,
        'focal_length': intrinsics[0, 0] if intrinsics is not None else None
    }


def predict_segmentation(image_rgb, score_threshold=0.5):
    """
    Предсказывает сегментацию объектов.
    """
    if isinstance(image_rgb, np.ndarray):
        image_tensor = torch.as_tensor(image_rgb, dtype=torch.float32).permute(2, 0, 1) / 255.0
    else:
        image_tensor = torch.as_tensor(np.array(image_rgb), dtype=torch.float32).permute(2, 0, 1) / 255.0
    
    with torch.no_grad():
        prediction = segmentation_model([image_tensor.to(DEVICE)])[0]
    
    keep = prediction['scores'] >= score_threshold
    filtered = {
        'boxes': prediction['boxes'][keep].cpu(),
        'labels': prediction['labels'][keep].cpu(),
        'scores': prediction['scores'][keep].cpu(),
        'masks': prediction['masks'][keep].cpu()
    }
    
    return filtered


def predict_associations(boxes, labels, scores):
    """
    Предсказывает связи между параллелепипедами и кубами.
    """
    boxes_np = boxes.numpy() if isinstance(boxes, torch.Tensor) else boxes
    labels_np = labels.numpy() if isinstance(labels, torch.Tensor) else labels
    
    cube_indices = np.where(labels_np == 1)[0]
    para_indices = np.where(labels_np == 2)[0]
    
    associations = {}
    
    for para_idx in para_indices:
        para_box = boxes_np[para_idx]
        para_center_x = (para_box[0] + para_box[2]) / 2
        para_bottom = para_box[3]
        
        min_dist = float('inf')
        best_cube_idx = None
        
        for cube_idx in cube_indices:
            cube_box = boxes_np[cube_idx]
            cube_center_x = (cube_box[0] + cube_box[2]) / 2
            cube_top = cube_box[1]
            
            vertical_dist = abs(para_bottom - cube_top)
            horizontal_dist = abs(para_center_x - cube_center_x)
            
            dist = np.sqrt(vertical_dist**2 + horizontal_dist**2)
            
            if cube_box[0] <= para_center_x <= cube_box[2]:
                dist *= 0.5
            
            if dist < min_dist:
                min_dist = dist
                best_cube_idx = cube_idx
        
        associations[int(para_idx)] = int(best_cube_idx) if best_cube_idx is not None else None
    
    return associations

In [ ]:
def calculate_object_distance(depth_map, confidence_map, mask, boxes, idx, conf_threshold=0.3):
    """
    Вычисляет расстояние до объекта с учётом confidence map.
    
    Args:
        depth_map: (H, W) depth в метрах
        confidence_map: (H, W) confidence (0-1) или None
        mask: маски объектов
        boxes: bounding boxes
        idx: индекс объекта
        conf_threshold: минимальная confidence
    
    Returns:
        distance_info: dict с расстояниями и confidence
    """
    if isinstance(mask, torch.Tensor):
        obj_mask = mask[idx, 0].numpy() > 0.5
    else:
        obj_mask = mask > 0.5
    
    # Применяем confidence фильтр если есть
    if confidence_map is not None:
        high_conf_mask = confidence_map >= conf_threshold
        obj_mask = obj_mask & high_conf_mask
    
    object_depths = depth_map[obj_mask]
    
    if len(object_depths) == 0:
        return {
            'min': 0.0,
            'max': 0.0,
            'mean': 0.0,
            'median': 0.0,
            'center': 0.0,
            'confidence': 0.0,
            'valid_pixels': 0
        }
    
    # Расстояние до центра bbox
    box = boxes[idx].numpy() if isinstance(boxes[idx], torch.Tensor) else boxes[idx]
    center_y = int((box[1] + box[3]) / 2)
    center_x = int((box[0] + box[2]) / 2)
    center_depth = depth_map[center_y, center_x]
    
    # Средняя confidence для объекта
    avg_conf = 1.0
    if confidence_map is not None:
        obj_mask_full = mask[idx, 0].numpy() > 0.5 if isinstance(mask, torch.Tensor) else mask > 0.5
        avg_conf = float(confidence_map[obj_mask_full].mean())
    
    return {
        'min': float(np.min(object_depths)),
        'max': float(np.max(object_depths)),
        'mean': float(np.mean(object_depths)),
        'median': float(np.median(object_depths)),
        'center': float(center_depth),
        'confidence': avg_conf,
        'valid_pixels': len(object_depths)
    }

## 5. Полный inference pipeline

In [ ]:
def full_inference(image_path, visualize=True):
    """
    Полный pipeline с Depth Anything V3.
    """
    # 1. Загрузка изображения
    image_rgb = np.array(Image.open(image_path).convert('RGB'))
    
    print(f"📸 Processing: {os.path.basename(image_path)}")
    
    # 2. Depth estimation с V3
    print("   🔍 Estimating depth with V3...")
    depth_result = predict_depth_v3(image_rgb)
    
    depth_map = depth_result['depth']
    confidence_map = depth_result['confidence']
    intrinsics = depth_result['intrinsics']
    focal_length = depth_result['focal_length']
    
    print(f"      Depth range: {depth_map.min():.2f}m - {depth_map.max():.2f}m")
    if focal_length is not None:
        print(f"      Estimated focal length: {focal_length:.2f}px (expected: {EXPECTED_FOCAL_LENGTH:.2f}px)")
        error_pct = abs(focal_length - EXPECTED_FOCAL_LENGTH) / EXPECTED_FOCAL_LENGTH * 100
        print(f"      Focal length error: {error_pct:.1f}%")
    
    # 3. Segmentation
    print("   🎯 Segmenting objects...")
    segmentation = predict_segmentation(image_rgb)
    
    # 4. Associations
    print("   🔗 Predicting associations...")
    associations = predict_associations(
        segmentation['boxes'],
        segmentation['labels'],
        segmentation['scores']
    )
    
    # 5. Calculate distances
    print("   📏 Calculating distances...")
    distances = []
    for i in range(len(segmentation['labels'])):
        dist_info = calculate_object_distance(
            depth_map,
            confidence_map,
            segmentation['masks'],
            segmentation['boxes'],
            i,
            conf_threshold=CONFIDENCE_THRESHOLD
        )
        distances.append(dist_info)
    
    # 6. Summary
    n_cubes = (segmentation['labels'] == 1).sum().item()
    n_paras = (segmentation['labels'] == 2).sum().item()
    
    print(f"\n   ✅ Results:")
    print(f"      Cubes: {n_cubes}")
    print(f"      Parallelepipeds: {n_paras}")
    print(f"      Associations: {len(associations)}")
    
    # 7. Visualization
    if visualize:
        visualize_results(
            image_rgb,
            depth_map,
            confidence_map,
            segmentation,
            associations,
            distances,
            intrinsics
        )
    
    return {
        'image': image_rgb,
        'depth_map': depth_map,
        'confidence_map': confidence_map,
        'intrinsics': intrinsics,
        'segmentation': segmentation,
        'associations': associations,
        'distances': distances
    }

## 6. Визуализация

In [ ]:
def visualize_results(image, depth_map, confidence_map, segmentation, associations, distances, intrinsics):
    """
    Визуализирует все результаты включая confidence map.
    """
    fig = plt.figure(figsize=(20, 15))
    gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)
    
    boxes = segmentation['boxes'].numpy()
    labels = segmentation['labels'].numpy()
    scores = segmentation['scores'].numpy()
    masks = segmentation['masks'].numpy()
    
    # 1. Original RGB
    ax1 = fig.add_subplot(gs[0, 0])
    ax1.imshow(image)
    ax1.set_title('Original Image', fontsize=14, fontweight='bold')
    ax1.axis('off')
    
    # 2. Depth Map
    ax2 = fig.add_subplot(gs[0, 1])
    depth_vis = ax2.imshow(depth_map, cmap='turbo')
    ax2.set_title(f'Depth Map (V3)\nRange: {depth_map.min():.2f}m - {depth_map.max():.2f}m', 
                  fontsize=14, fontweight='bold')
    ax2.axis('off')
    plt.colorbar(depth_vis, ax=ax2, fraction=0.046, pad=0.04, label='Depth (m)')
    
    # 3. Confidence Map
    ax3 = fig.add_subplot(gs[0, 2])
    if confidence_map is not None:
        conf_vis = ax3.imshow(confidence_map, cmap='viridis', vmin=0, vmax=1)
        ax3.set_title(f'Confidence Map\nMean: {confidence_map.mean():.3f}', 
                      fontsize=14, fontweight='bold')
        plt.colorbar(conf_vis, ax=ax3, fraction=0.046, pad=0.04, label='Confidence')
    else:
        ax3.text(0.5, 0.5, 'No confidence map', ha='center', va='center')
        ax3.set_title('Confidence Map', fontsize=14, fontweight='bold')
    ax3.axis('off')
    
    # 4. Segmentation with distances
    ax4 = fig.add_subplot(gs[1, :])
    ax4.imshow(image)
    
    colors = {1: 'red', 2: 'green'}
    
    for i in range(len(labels)):
        box = boxes[i]
        label = labels[i]
        dist = distances[i]
        color = colors.get(label, 'blue')
        
        # Draw bbox
        from matplotlib.patches import Rectangle
        rect = Rectangle(
            (box[0], box[1]), box[2] - box[0], box[3] - box[1],
            fill=False, edgecolor=color, linewidth=2
        )
        ax4.add_patch(rect)
        
        # Draw label with distance and confidence
        text = f"{CATEGORIES[label]}\n{dist['center']:.2f}m\nconf: {dist['confidence']:.2f}"
        ax4.text(
            box[0], box[1] - 10,
            text,
            color='white',
            fontsize=9,
            bbox=dict(boxstyle='round', facecolor=color, alpha=0.8)
        )
    
    ax4.set_title('Segmentation + Distances + Confidence', fontsize=14, fontweight='bold')
    ax4.axis('off')
    
    # 5. Masks with associations
    ax5 = fig.add_subplot(gs[2, :])
    ax5.imshow(image)
    
    # Draw masks
    combined_mask = np.zeros((*image.shape[:2], 3))
    for i in range(len(labels)):
        mask = masks[i, 0] > 0.5
        label = labels[i]
        
        if label == 1:
            combined_mask[mask] = [1, 0, 0]
        elif label == 2:
            combined_mask[mask] = [0, 1, 0]
    
    ax5.imshow(combined_mask, alpha=0.5)
    
    # Draw association lines
    for para_idx, cube_idx in associations.items():
        if cube_idx is not None:
            para_box = boxes[para_idx]
            cube_box = boxes[cube_idx]
            
            para_center = [(para_box[0] + para_box[2]) / 2, (para_box[1] + para_box[3]) / 2]
            cube_center = [(cube_box[0] + cube_box[2]) / 2, (cube_box[1] + cube_box[3]) / 2]
            
            ax5.plot(
                [para_center[0], cube_center[0]],
                [para_center[1], cube_center[1]],
                'y-', linewidth=2
            )
            ax5.plot(*para_center, 'yo', markersize=8)
            ax5.plot(*cube_center, 'yo', markersize=8)
    
    ax5.set_title('Masks + Associations', fontsize=14, fontweight='bold')
    ax5.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Print detailed info
    print("\n📊 Detailed Information:")
    print("=" * 90)
    
    if intrinsics is not None:
        print(f"\n📷 Estimated Camera Intrinsics:")
        print(f"   Focal Length X: {intrinsics[0, 0]:.2f}px")
        print(f"   Focal Length Y: {intrinsics[1, 1]:.2f}px")
        print(f"   Principal Point: ({intrinsics[0, 2]:.2f}, {intrinsics[1, 2]:.2f})")
    
    print(f"\n📏 Object Distances:")
    for i in range(len(labels)):
        label_name = CATEGORIES[labels[i]]
        dist = distances[i]
        parent_info = ""
        
        if i in associations:
            parent_idx = associations[i]
            if parent_idx is not None:
                parent_dist = distances[parent_idx]
                parent_info = f" → Parent cube at {parent_dist['center']:.2f}m"
        
        print(f"[{i}] {label_name}:")
        print(f"    Center: {dist['center']:.3f}m | Mean: {dist['mean']:.3f}m | "
              f"Range: {dist['min']:.3f}m - {dist['max']:.3f}m")
        print(f"    Confidence: {dist['confidence']:.3f} | Valid pixels: {dist['valid_pixels']}{parent_info}")
    print("=" * 90)

## 7. Запуск inference

In [ ]:
import glob

image_files = sorted(glob.glob(os.path.join(IMAGES_PATH, "*.png")))[:5]

print(f"Found {len(image_files)} images for inference\n")

all_results = []

for img_path in image_files:
    print("\n" + "="*90)
    results = full_inference(img_path, visualize=True)
    all_results.append(results)
    print("="*90)

## 8. Статистика и сравнение с Unity

In [ ]:
# Собираем статистику
all_cube_distances = []
all_para_distances = []
all_focal_lengths = []

for result in all_results:
    labels = result['segmentation']['labels'].numpy()
    distances = result['distances']
    
    for i, label in enumerate(labels):
        if label == 1:
            all_cube_distances.append(distances[i]['center'])
        elif label == 2:
            all_para_distances.append(distances[i]['center'])
    
    if result['intrinsics'] is not None:
        all_focal_lengths.append(result['intrinsics'][0, 0])

# Визуализация
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Distance distributions
axes[0].hist(all_cube_distances, bins=20, color='red', alpha=0.7, edgecolor='black')
axes[0].set_title('Distance Distribution: Red Cubes', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Distance (m)')
axes[0].set_ylabel('Count')
axes[0].grid(True, alpha=0.3)

axes[1].hist(all_para_distances, bins=20, color='green', alpha=0.7, edgecolor='black')
axes[1].set_title('Distance Distribution: Green Parallelepipeds', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Distance (m)')
axes[1].set_ylabel('Count')
axes[1].grid(True, alpha=0.3)

# Focal length comparison
if all_focal_lengths:
    axes[2].hist(all_focal_lengths, bins=15, color='blue', alpha=0.7, edgecolor='black')
    axes[2].axvline(EXPECTED_FOCAL_LENGTH, color='red', linestyle='--', linewidth=2, 
                    label=f'Unity Expected: {EXPECTED_FOCAL_LENGTH:.1f}px')
    axes[2].set_title('Estimated Focal Length Distribution', fontsize=14, fontweight='bold')
    axes[2].set_xlabel('Focal Length (pixels)')
    axes[2].set_ylabel('Count')
    axes[2].legend()
    axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n📈 Overall Statistics:")
print(f"\n   Cubes: {len(all_cube_distances)} detected")
print(f"      Mean distance: {np.mean(all_cube_distances):.3f}m")
print(f"      Std: {np.std(all_cube_distances):.3f}m")
print(f"      Range: {np.min(all_cube_distances):.3f}m - {np.max(all_cube_distances):.3f}m")

print(f"\n   Parallelepipeds: {len(all_para_distances)} detected")
print(f"      Mean distance: {np.mean(all_para_distances):.3f}m")
print(f"      Std: {np.std(all_para_distances):.3f}m")
print(f"      Range: {np.min(all_para_distances):.3f}m - {np.max(all_para_distances):.3f}m")

if all_focal_lengths:
    print(f"\n   📷 Focal Length Estimation:")
    print(f"      Mean estimated: {np.mean(all_focal_lengths):.2f}px")
    print(f"      Unity expected: {EXPECTED_FOCAL_LENGTH:.2f}px")
    print(f"      Error: {abs(np.mean(all_focal_lengths) - EXPECTED_FOCAL_LENGTH):.2f}px "
          f"({abs(np.mean(all_focal_lengths) - EXPECTED_FOCAL_LENGTH) / EXPECTED_FOCAL_LENGTH * 100:.1f}%)")

## 9. Экспорт результатов

In [ ]:
output_results = []

for i, (img_path, result) in enumerate(zip(image_files, all_results)):
    labels = result['segmentation']['labels'].numpy()
    boxes = result['segmentation']['boxes'].numpy()
    scores = result['segmentation']['scores'].numpy()
    distances = result['distances']
    associations = result['associations']
    intrinsics = result['intrinsics']
    
    objects = []
    for j in range(len(labels)):
        obj = {
            'id': j,
            'category': CATEGORIES[labels[j]],
            'bbox': boxes[j].tolist(),
            'score': float(scores[j]),
            'distance': {
                'center': distances[j]['center'],
                'mean': distances[j]['mean'],
                'min': distances[j]['min'],
                'max': distances[j]['max'],
                'confidence': distances[j]['confidence']
            }
        }
        
        if j in associations:
            obj['parent_id'] = associations[j]
        
        objects.append(obj)
    
    result_dict = {
        'image': os.path.basename(img_path),
        'depth_range': {
            'min': float(result['depth_map'].min()),
            'max': float(result['depth_map'].max()),
            'mean': float(result['depth_map'].mean())
        },
        'objects': objects
    }
    
    if intrinsics is not None:
        result_dict['camera_intrinsics'] = {
            'focal_length_x': float(intrinsics[0, 0]),
            'focal_length_y': float(intrinsics[1, 1]),
            'principal_point_x': float(intrinsics[0, 2]),
            'principal_point_y': float(intrinsics[1, 2])
        }
    
    output_results.append(result_dict)

with open('depth_v3_segmentation_results.json', 'w') as f:
    json.dump(output_results, f, indent=2)

print("✅ Results saved to depth_v3_segmentation_results.json")

## 📝 Резюме: Depth Anything V3

### Что делает этот notebook:

1. **Depth Estimation с V3**: Автоматически оценивает camera intrinsics и depth в метрах
2. **Confidence Map**: Фильтрует ненадёжные области depth
3. **Segmentation**: Mask R-CNN для кубов и параллелепипедов
4. **Associations**: Определение связей между объектами
5. **Distance Calculation**: Точные расстояния с учётом confidence

### Преимущества V3 над V2:

- ✅ **Автоопределение focal length** - не нужно передавать из Unity!
- ✅ **Depth сразу в метрах** (для Nested моделей)
- ✅ **Confidence map** для фильтрации
- ✅ **Лучшая точность** на indoor сценах
- ✅ **Multi-view support** (опционально)

### Применение:

- Робототехника (навигация, захват объектов)
- AR/VR (размещение виртуальных объектов)
- Автономные системы (определение препятствий)
- 3D реконструкция (с multi-view)
- Контроль качества (измерение размеров)